# Assignment 1: This is the title of the assignment

Authors:  
Shayan Paydar Darian and 
Daniel Pazirai 


## Imports

In [1]:
# It is recommended to start with general import statements
from utility_functions import *
import pandas as pd
import transformers
import datasets
import torch
import os

print(pd.__version__)
print(transformers.__version__)
print(datasets.__version__)
print(torch.__version__)

c:\Users\Shayan\anaconda3\envs\swedish-ner\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.2.3
4.49.0
3.4.0
2.5.1


## Load data

This section should load the raw dataset for the task.  
Remember to use relative paths to load any files in the notebook.

In [ ]:
# Load the Swedish Medical NER dataset
import pandas as pd

df_1177 = pd.read_parquet("hf://datasets/community-datasets/swedish_medical_ner/1177/train-00000-of-00001.parquet")

df_lt = pd.read_parquet("hf://datasets/community-datasets/swedish_medical_ner/lt/train-00000-of-00001.parquet")

df_wiki = pd.read_parquet("hf://datasets/community-datasets/swedish_medical_ner/wiki/train-00000-of-00001.parquet")

print(df_1177.head)
print(df_lt.head)
print(df_wiki.head)

<bound method NDFrame.head of           sid                                           sentence  \
0      1177_0  Memantin ( Ebixa ) ger sällan några biverkningar.   
1      1177_1  Det är också lättare att dosera [ flytande med...   
2      1177_2  ( Förstoppning ) är ett vanligt problem hos äl...   
3      1177_3  [ Medicinen ] kan också göra att man blöder lä...   
4      1177_4  Barn har större möjligheter att samarbeta om d...   
..        ...                                                ...   
922  1177_922  Ofta kan man ha flera besvär i olika delar av ...   
923  1177_923  Vaccinationen ger inte ett fullständigt skydd ...   
924  1177_924  Vissa personer kan märka av klara förbättringa...   
925  1177_925  ( Demens ) innebär dels problem med minnet och...   
926  1177_926                                                      

                                              entities  
0    {'start': [9], 'end': [18], 'text': ['Ebixa'],...  
1    {'start': [32], 'end': [52], 'text

## Save the raw data

In [ ]:
os.makedirs('raw_data', exist_ok=True)

# Save the DataFrames as parquet files
df_1177.to_parquet('raw_data/df_1177.parquet')
df_lt.to_parquet('raw_data/df_lt.parquet')
df_wiki.to_parquet('raw_data/df_wiki.parquet')

In [9]:
print(df_1177.info())
print(df_lt.info())
print(df_wiki.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927 entries, 0 to 926
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sid       927 non-null    object
 1   sentence  927 non-null    object
 2   entities  927 non-null    object
dtypes: object(3)
memory usage: 21.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745753 entries, 0 to 745752
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sid       745753 non-null  object
 1   sentence  745753 non-null  object
 2   entities  745753 non-null  object
dtypes: object(3)
memory usage: 17.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48720 entries, 0 to 48719
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sid       48720 non-null  object
 1   sentence  48720 non-null  object
 2   entities  48720 non-null  object
dtypes: obj

## Task 1: Survey (kolla upp vilka modeller som finns och kolla vilken LLN som är bäst för Swedish Biomedical NER)

## Survey of Models for Swedish Biomedical NER

| **Model**        | **Language(s)** | **Strengths**                                   | **Weaknesses**                                  | **Relevance for Biomedical NER (Swedish)**               |
|------------------|-----------------|-------------------------------------------------|-------------------------------------------------|----------------------------------------------------------|
| **KB-BERT**      | Swedish         | Native Swedish, decent at general NER tasks     | No biomedical pretraining                       | Good starting point                                      |
| **Swedish ALBERT** | Swedish       | Lightweight, faster to train                    | Lacks domain specialization                    | OK, but may miss nuances                                 |
| **mBERT**        | Multilingual    | Cross-lingual abilities                         | Not optimized for any specific language/domain | Medium                                                   |
| **XLM-RoBERTa**  | Multilingual    | Robust multilingual understanding               | Same as mBERT, no biomedical specialization    | Medium                                                   |
| **BioBERT (English)** | English    | Biomedical domain specialized                   | Needs translated data, loses Swedish nuances   | Very good if translation is perfect                      |
 
 The survey results are based on searches on HuggingFace and google scholar with keywords (Swedish NER, Biomedical NER, etc), in order to see what models there are and what models have been used before.


## Approach and Motivation
In our approach, we want to use a Swedish Language-Model rather than translating the existing data into English. Although models like BioBERT offer very good performance in biomedical NER tasks, translation of clinical text can introduce errors and result in a loss of contextual information. This is particularly risky in the biomedical domain, where subtle differences in terminology can significantly impact the meaning of text. Additionally, as we are not biomedical experts, manually validating translations would be challenging.

Therefore, our approach is to fine-tune an existing Swedish model for biomedical NER, using the Swedish Medical NER dataset provided. 

The three models found on HuggingFace which were relevant were: 

bert-base-swedish-cased (v1) - A BERT trained with the same hyperparameters as first published by Google.

bert-base-swedish-cased-ner (experimental) - a BERT fine-tuned for NER using SUC 3.0.

albert-base-swedish-cased-alpha (alpha) - A first attempt at an ALBERT for Swedish.

## Our Choice
Out of of these models, we will be using the base model (bert-base-swedish-cased (v1)). Although the experimental version is fine tuned for NER, we want to train it on specifically biomedical NER. The base model will be perfect for that since it's flexible and gives us more control of fine tuning it and has less prior bias. We will also not use the Albert version since it's a first attempt at it and still in alpha.

Link for the KB bert models: https://huggingface.co/KB/bert-base-swedish-cased-ner 

Study which used KB/bert-base-swedish-cased and
bert-base-multilingual-cased: (Although not related to biomedicine) https://aclanthology.org/2024.caldpseudo-1.7.pdf 

It is mandatory to maintain the headings for each task.  
OPTIONALLY, you can use one level down (###) to organize subsessions of the assignments.

Use markdown cells like this one to include:
- Discussion points.
- References to specific sources of code that you might have used to solve the assignment.
- General commentas and explanations about your solution.

In [ ]:
# Always use comments in the code to document specific steps

## Task 2: Välj modell, ladda ner dataset som står (https://huggingface.co/datasets/community-datasets/swedish_medical_ner) (laddat det med imports)

This section should contain the solution of task 2.

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="KB/bert-base-swedish-cased")

# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("KB/bert-base-swedish-cased")
model = AutoModelForTokenClassification.from_pretrained("KB/bert-base-swedish-cased")

c:\Users\Shayan\anaconda3\envs\swedish-ner\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shayan\.cache\huggingface\hub\models--KB--bert-base-swedish-cased-ner. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at KB/bert-base-swedish-cased-ner were not used when init

In [7]:
# Save model:

# Where you want to save it
output_dir = "./models/my-swedish-ner-model-untrained"

# Save model
model.save_pretrained(output_dir)

# Save tokenizer (important!)
tokenizer.save_pretrained(output_dir)

('./models/my-swedish-ner-model-untrained\\tokenizer_config.json',
 './models/my-swedish-ner-model-untrained\\special_tokens_map.json',
 './models/my-swedish-ner-model-untrained\\vocab.txt',
 './models/my-swedish-ner-model-untrained\\added_tokens.json',
 './models/my-swedish-ner-model-untrained\\tokenizer.json')

## Task 3: 

Preprocess the Dataset for Token Classification
Most likely, the data will be in token-label format. HuggingFace NER models expect:

Input IDs
Attention Masks
Labels per token
I can walk you through the actual code, but at a high level:

Tokenize the sentences.
Align the labels with the tokens.
Format into a HuggingFace dataset.

In [ ]:
from datasets import Dataset, concatenate_datasets

# Convert pandas DataFrames to Hugging Face Datasets
dataset_1177 = Dataset.from_pandas(df_1177)
dataset_lt = Dataset.from_pandas(df_lt)
dataset_wiki = Dataset.from_pandas(df_wiki)

# Combine all three into one big dataset (optional)
dataset = concatenate_datasets([dataset_1177, dataset_lt, dataset_wiki])

## Task 4:  Load the Model and Tokenizer och Fine-Tunea modellen

## Task 5: Evaluera och kolla var den failade, vilka resultat hade den? etc

## Results and Discussion

This section should contain:
- Results.
- Summary of best model performance:
    - Name of best model file as saved in /models.
    - Relevant scores such as: accuracy, precision, recall, F1-score, etc.
- Key discussion points.

In [ ]:
# Always use comments in the code to document specific steps